In [ ]:
import os
import tempfile
import warnings
from os.path import join as pjoin

import dask
import dask.dataframe as dd
import dask_geopandas as dgpd
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio
from scipy.fft import dst
import tqdm
import xarray as xr
from dask.diagnostics import ProgressBar
from rasterio.crs import CRS

from raster_tools import Raster, Vector, open_vectors, clipping, zonal
from raster_tools.dtypes import F32, U8

import matplotlib.pyplot as plt

In [ ]:
# Location for temporary storage
TMP_LOC = "/home/jake/FireLab/Project/data/temp/"
DATA_LOC = "/home/jake/FireLab/Project/data/"

STATE = "OR"

# Location of clipped DEM files
DEM_DATA_DIR = pjoin(TMP_LOC, "dem_data")

# location of feature data files
FEATURE_DIR = pjoin(DATA_LOC, "FeatureData")
EDNA_DIR = pjoin(DATA_LOC, "terrain")
MTBS_DIR = pjoin(DATA_LOC, "MTBS_Data")

mtbs_df_path = pjoin(TMP_LOC, f"{STATE}_mtbs.parquet")
mtbs_df_temp_path = pjoin(TMP_LOC, f"{STATE}_mtbs_temp.parquet")
checkpoint_1_path = pjoin(TMP_LOC, "check1")
checkpoint_2_path = pjoin(TMP_LOC, "check2")

PATHS = {
    "states": pjoin(EDNA_DIR, "state_borders/cb_2018_us_state_5m.shp"),
    "dem": pjoin(EDNA_DIR, "us_orig_dem/us_orig_dem/orig_dem/hdr.adf"),
    "dem_slope": pjoin(EDNA_DIR, "us_slope/us_slope/slope/hdr.adf"),
    "dem_aspect": pjoin(EDNA_DIR, "us_aspect/aspect/hdr.adf"),
    "dem_flow_acc": pjoin(EDNA_DIR, "us_flow_acc/us_flow_acc/flow_acc/hdr.adf"),
    "gm_srad": pjoin(FEATURE_DIR, "gridmet/srad_1986_2020_weekly.nc"),
    "gm_vpd": pjoin(FEATURE_DIR, "gridmet/vpd_1986_2020_weekly.nc"),
    "aw_mat": pjoin(FEATURE_DIR, "adaptwest/Normal_1991_2020_MAT.tif"),
    "aw_mcmt": pjoin(FEATURE_DIR, "adaptwest/Normal_1991_2020_MCMT.tif"),
    "aw_mwmt": pjoin(FEATURE_DIR, "adaptwest/Normal_1991_2020_MWMT.tif"),
    "aw_td": pjoin(FEATURE_DIR, "adaptwest/Normal_1991_2020_TD.tif"),
    "dm_tmax": pjoin(FEATURE_DIR, "daymet/tmax_1986_2020.nc"),
    "dm_tmin": pjoin(FEATURE_DIR, "daymet/tmin_1986_2020.nc"),
    "biomass_afg": pjoin(
        FEATURE_DIR, "biomass/biomass_afg_1986_2020_{}.nc".format(STATE)
    ),
    "biomass_pfg": pjoin(
        FEATURE_DIR, "biomass/biomass_pfg_1986_2020_{}.nc".format(STATE)
    ),
    "landfire_fvt": pjoin(
        FEATURE_DIR, "landfire/LF2020_FVT_200_CONUS/Tif/LC20_FVT_200.tif"
    ),
    "landfire_fbfm40": pjoin(
        FEATURE_DIR, "landfire/LF2020_FBFM40_200_CONUS/Tif/LC20_F40_200.tif"
    ),
    "ndvi": pjoin(FEATURE_DIR, "ndvi/access/weekly/ndvi_1986_2020_weekavg.nc"),
    "mtbs_root": pjoin(MTBS_DIR, "MTBS_BSmosaics/"),
    "mtbs_perim": pjoin(MTBS_DIR, "mtbs_perimeter_data/mtbs_perims_DD.shp"),
}
YEARS = list(range(1986, 2021))
GM_KEYS = list(filter(lambda x: x.startswith("gm_"), PATHS))
AW_KEYS = list(filter(lambda x: x.startswith("aw_"), PATHS))
DM_KEYS = list(filter(lambda x: x.startswith("dm_"), PATHS))
BIOMASS_KEYS = list(filter(lambda x: x.startswith("biomass_"), PATHS))
LANDFIRE_KEYS = list(filter(lambda x: x.startswith("landfire_"), PATHS))
NDVI_KEYS = list(filter(lambda x: x.startswith("ndvi"), PATHS))
DEM_KEYS = list(filter(lambda x: x.startswith("dem"), PATHS))

In [ ]:
with ProgressBar():
    df = dgpd.read_parquet(checkpoint_2_path)

In [ ]:
nc_ds = xr.open_dataset(PATHS["aw_mat"], chunks={"day": 1})

In [ ]:
nc_ds.variables

In [ ]:
# a function for comparing and displaying data / coords / info from a collection of xarray datasets
def compare_xr_datasets(datasets, attrs=False, coords=False, info=False):
    for ds in datasets:
        print(f"{ds = }")
        if attrs:
            print(f"{ds.attrs = }")
        if coords:
            print(f"{ds.coords = }")
        if info:
            print(f"{ds.info = }")
        print("-----------------\n\n")

nc_datasets = [xr.open_dataset(PATHS['dm_tmax'], chunks={"day": 1}), xr.open_dataset(PATHS['aw_mat'], chunks={"day": 1})]
compare_xr_datasets(nc_datasets, attrs=True, coords=True, info=True)

In [ ]:
nc_ds = xr.open_dataset(PATHS['dm_tmax'], chunks={"day": 1})#, decode_times=False)
nc_ds = nc_ds.rio.write_crs("EPSG:5071")  # FOR DAYMET ONLY!!
nc_ds = nc_ds.rio.write_crs(
    nc_ds.coords["lambert_conformal_conic"].spatial_ref
)  # FOR DAYMET ONLY!!
nc_ds = nc_ds.rename({"lambert_conformal_conic": "crs"})  # FOR DAYMET ONLY!!
nc_ds2 = nc_ds.drop_vars(["lat", "lon"])  # FOR DAYMET ONLY!!
nc_ds2 = nc_ds2.rename_vars({"x": "lon", "y": "lat"})  # FOR DAYMET ONLY!!
nc_ds2

In [ ]:
df.ig_date.value_counts()

In [ ]:
# visually map/draw the fire from df with ig_date = 1986-03-20
import matplotlib.pyplot as plt
# plot with oregon map as background
fig, ax = plt.subplots(figsize=(15,15))
states = gpd.read_file(PATHS['states'])
states = states.to_crs(df.crs)
# limit states to just oregon
states = states[states.STUSPS == 'OR']
states.plot(ax=ax, color='white', edgecolor='black')
df[df.ig_date == '2014-07-14'].plot(ax=ax, color='red')
plt.show()

In [5]:
df = dgpd.read_parquet(mtbs_df_path)

In [14]:
df.compute().shape

55328732

In [ ]:
ndvi_ds = xr.open_dataset(PATHS['ndvi'], chunks={"day": 1})

In [ ]:
ndvi_ds.crs

In [ ]:
dataset = dgpd.read_parquet('data/temp/old_OR_mtbs.parquet/')

In [ ]:
dataset.head()

In [ ]:
year_to_mtbs_file = {
        y: pjoin(PATHS["mtbs_root"], f"mtbs_{STATE}_{y}.tif")
        for y in YEARS
    }

In [ ]:
year_to_mtbs_file

In [ ]:
# MTBS Perimeters
    print("Loading MTBS perimeters")
    perimdf = open_vectors(PATHS["mtbs_perim"]).data.to_crs("EPSG:5071")
    state_fire_perims = perimdf.clip(state_shape.compute())
    state_fire_perims = (
        state_fire_perims.assign(
            Ig_Date=lambda frame: dd.to_datetime(
                frame.Ig_Date, format="%Y-%m-%d"
            )
        )
        .sort_values("Ig_Date")
        .compute()
    )
    year_to_perims = {
        y: state_fire_perims[state_fire_perims.Ig_Date.dt.year == y]
        for y in YEARS
    }